In [45]:
# ================================
# 1. Importation des Bibliothèques
# ================================
import os
import cv2
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from datetime import datetime
from collections import defaultdict

In [46]:
# ================================
# 2. Répertoires et Variables Globales
# ================================
input_dir = '../data/raw/landsat'
cropped_dir = '../data/raw/cropped_straightened'
ndvi_output_dir = '../ndvi/ndvi_cropped_straightened_images'
os.makedirs(cropped_dir, exist_ok=True)
os.makedirs(ndvi_output_dir, exist_ok=True)

ndvi_labels = [
    "Eau", "Sols Nus/Urbain", "Végétation Très Faible", 
    "Végétation Faible", "Végétation Modérée", 
    "Végétation Dense", "Forêt Tropicale"
]

In [47]:
# ================================
# 3. Chargement et Prétraitement des Images
# ================================
def load_bands(red_band_path, nir_band_path):
    """ Charger les bandes avec rasterio """
    with rasterio.open(red_band_path) as red_src:
        red = red_src.read(1).astype('float64')
        red_meta = red_src.meta
    
    with rasterio.open(nir_band_path) as nir_src:
        nir = nir_src.read(1).astype('float64')
    
    return red, nir, red_meta

In [48]:
# ================================
# 4. Cropping et Redressement des Images
# ================================
def order_points(pts):
    """ Trie les points dans l'ordre: haut-gauche, haut-droit, bas-droit, bas-gauche """
    rect = np.zeros((4, 2), dtype="float32")
    
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    
    return rect

def detect_and_straighten(image, date_str, band_name):
    """ Détecte les contours et redresse l'image """
    norm_image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    _, thresholded = cv2.threshold(norm_image, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea)
    
    epsilon = 0.02 * cv2.arcLength(largest_contour, True)
    approx = cv2.approxPolyDP(largest_contour, epsilon, True)
    
    if len(approx) >= 4:
        pts = approx.reshape(-1, 2)
        rect = order_points(pts)
        
        (tl, tr, br, bl) = rect
        widthA = np.linalg.norm(br - bl)
        widthB = np.linalg.norm(tr - tl)
        maxWidth = max(int(widthA), int(widthB))
        
        heightA = np.linalg.norm(tr - br)
        heightB = np.linalg.norm(tl - bl)
        maxHeight = max(int(heightA), int(heightB))
        
        dst = np.array([
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]
        ], dtype="float32")
        
        M = cv2.getPerspectiveTransform(rect, dst)
        straightened_image = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
        
        # Sauvegarder l'image croppée et redressée
        output_path = os.path.join(cropped_dir, f"{date_str}_{band_name}_cropped_straightened.TIF")
        with rasterio.open(
            output_path,
            'w',
            driver='GTiff',
            height=straightened_image.shape[0],
            width=straightened_image.shape[1],
            count=1,
            dtype='float32',
            crs=None
        ) as dst:
            dst.write(straightened_image, 1)
        
        return straightened_image
    else:
        print("Les coins n'ont pas été correctement détectés.")
        return image

In [49]:
# ================================
# 5. Recadrage pour Uniformisation des Dimensions
# ================================
def crop_to_same_size(image1, image2):
    """ Recadre les deux images à la même taille """
    min_rows = min(image1.shape[0], image2.shape[0])
    min_cols = min(image1.shape[1], image2.shape[1])
    
    image1_cropped = image1[:min_rows, :min_cols]
    image2_cropped = image2[:min_rows, :min_cols]
    
    return image1_cropped, image2_cropped

In [50]:
# ================================
# 6. Calcul et Sauvegarde du NDVI
# ================================
def calculate_ndvi(red, nir):
    """ Calcul du NDVI """
    ndvi = np.where((nir + red) == 0., 0, (nir - red) / (nir + red))
    return ndvi

def calculate_and_save_ndvi(red, nir, date_str, meta_data):
    """ Calculer et sauvegarder le NDVI """
    ndvi = calculate_ndvi(red, nir)
    
    # Chemin de sauvegarde
    save_path = os.path.join(ndvi_output_dir, f"{date_str}_ndvi.TIF")
    
    # Sauvegarde avec rasterio en format GeoTIFF
    with rasterio.open(
        save_path,
        'w',
        driver='GTiff',
        height=ndvi.shape[0],
        width=ndvi.shape[1],
        count=1,
        dtype='float32',
        crs=meta_data['crs'],
        transform=meta_data['transform']
    ) as dst:
        dst.write(ndvi, 1)
    
    print(f"NDVI sauvegardé : {save_path}")
    return ndvi

In [51]:
# ================================
# 7. Classification des NDVI
# ================================
def classify_ndvi(ndvi):
    """ Classification du NDVI en classes définies """
    classes = np.zeros_like(ndvi)
    classes = np.where((ndvi <= -0.1), 1, classes)     # Eau
    classes = np.where((ndvi > -0.1) & (ndvi <= 0), 2, classes)  # Sols Nus/Urbain
    classes = np.where((ndvi > 0) & (ndvi <= 0.2), 3, classes)   # Végétation Très Faible
    classes = np.where((ndvi > 0.2) & (ndvi <= 0.4), 4, classes) # Végétation Faible
    classes = np.where((ndvi > 0.4) & (ndvi <= 0.6), 5, classes) # Végétation Modérée
    classes = np.where((ndvi > 0.6) & (ndvi <= 0.8), 6, classes) # Végétation Dense
    classes = np.where((ndvi > 0.8), 7, classes)                 # Forêt Tropicale
    return classes.astype(int)  # Conversion explicite en entier

In [52]:
# ================================
# 8. Analyse Temporelle Complète
# ================================
def temporal_analysis(ndvi_series):
    """
    Réalisation de l'analyse temporelle sur les NDVI calculés :
    - Cartes temporelles
    - Carte de différence entre deux dates
    - Heatmap d'évolution des classes NDVI
    - Graphique de transition des classes NDVI
    - Analyse de tendance par classe NDVI
    """
    dates = sorted(ndvi_series.keys())
    
    # Uniformisation des dimensions des NDVI
    min_rows = min(ndvi.shape[0] for ndvi in ndvi_series.values())
    min_cols = min(ndvi.shape[1] for ndvi in ndvi_series.values())
    
    # Recadrer toutes les images NDVI à la plus petite dimension trouvée
    for date in dates:
        ndvi_series[date] = ndvi_series[date][:min_rows, :min_cols]
    
    # 1. Cartes temporelles du NDVI
    for date, ndvi in ndvi_series.items():
        plt.figure(figsize=(10, 8))
        plt.title(f'NDVI - {date.strftime("%Y-%m-%d")}')
        plt.imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
        plt.colorbar(label='Valeur NDVI')
        plt.savefig(os.path.join(ndvi_output_dir, f"ndvi_{date.strftime('%Y-%m-%d')}.png"))
        plt.close()

    # 2. Carte de différence NDVI entre deux dates consécutives
    for i in range(len(dates) - 1):
        date1, date2 = dates[i], dates[i + 1]
        diff = ndvi_series[date2] - ndvi_series[date1]
        
        plt.figure(figsize=(10, 8))
        plt.title(f'Différence NDVI - {date2.strftime("%Y-%m-%d")} vs {date1.strftime("%Y-%m-%d")}')
        plt.imshow(diff, cmap='bwr', vmin=-1, vmax=1)
        plt.colorbar(label='Différence NDVI')
        plt.savefig(os.path.join(ndvi_output_dir, f"ndvi_diff_{date2.strftime('%Y-%m-%d')}_vs_{date1.strftime('%Y-%m-%d')}.png"))
        plt.close()
    
    # 3. Heatmap de l'évolution temporelle des classes NDVI
    all_classes = []
    for date, ndvi in ndvi_series.items():
        classified_ndvi = classify_ndvi(ndvi)
        all_classes.append(classified_ndvi.flatten())

    all_classes = np.array(all_classes)
    
    # Correction du TypeError avec conversion en entier
    all_classes = all_classes.astype(int)
    
    class_counts = np.apply_along_axis(
        lambda x: np.bincount(x, minlength=len(ndvi_labels)), 
        axis=1, 
        arr=all_classes
    )
    
    plt.figure(figsize=(14, 8))
    sns.heatmap(class_counts.T, cmap='viridis', cbar_kws={'label': 'Nombre de pixels'})
    plt.yticks(np.arange(len(ndvi_labels)) + 0.5, ndvi_labels, rotation=0)
    plt.xlabel('Dates')
    plt.ylabel('Classes NDVI')
    plt.title('Évolution Temporelle des Classes NDVI')
    plt.savefig(os.path.join(ndvi_output_dir, "heatmap_classes_ndvi.png"))
    plt.close()

    # 4. Graphique de transition des classes NDVI
    transitions = defaultdict(lambda: defaultdict(int))
    for i in range(len(dates) - 1):
        date1, date2 = dates[i], dates[i + 1]
        class1 = classify_ndvi(ndvi_series[date1]).flatten()
        class2 = classify_ndvi(ndvi_series[date2]).flatten()
        
        for c1, c2 in zip(class1, class2):
            transitions[c1][c2] += 1
    
    transition_matrix = np.zeros((len(ndvi_labels), len(ndvi_labels)))
    for i in range(len(ndvi_labels)):
        for j in range(len(ndvi_labels)):
            transition_matrix[i, j] = transitions[i + 1][j + 1]
    
    # Correction du TypeError avec conversion en entier
    transition_matrix = transition_matrix.astype(int)
    
    plt.figure(figsize=(14, 10))
    sns.heatmap(transition_matrix, annot=True, fmt='d', cmap='Blues', 
                xticklabels=ndvi_labels, yticklabels=ndvi_labels)
    plt.xlabel('Classe suivante')
    plt.ylabel('Classe précédente')
    plt.title('Transition des Classes NDVI dans le Temps')
    plt.savefig(os.path.join(ndvi_output_dir, "transition_classes_ndvi.png"))
    plt.close()

In [53]:
# ================================
# 9. Fonction Principale (__main__)
# ================================
def process_images():
    """ Chargement, Cropping, Redressement et Calcul du NDVI """
    ndvi_series = {}
    for date_dir in sorted(glob(os.path.join(input_dir, '*'))):
        date_str = os.path.basename(date_dir)
        try:
            date_obj = datetime.strptime(date_str, '%Y-%m-%d')
        except ValueError:
            continue
        
        b3_path = glob(os.path.join(date_dir, '*_B3.TIF'))[0]
        b4_path = glob(os.path.join(date_dir, '*_B4.TIF'))[0]
        
        red, nir, meta_data = load_bands(b3_path, b4_path)
        red_cropped = detect_and_straighten(red, date_str, 'B3_Rouge')
        nir_cropped = detect_and_straighten(nir, date_str, 'B4_NIR')
        
        red_cropped, nir_cropped = crop_to_same_size(red_cropped, nir_cropped)
        ndvi = calculate_and_save_ndvi(red_cropped, nir_cropped, date_str, meta_data)
        ndvi_series[date_obj] = ndvi
    
    return ndvi_series

if __name__ == "__main__":
    ndvi_series = process_images()
    temporal_analysis(ndvi_series)
    print("Analyse temporelle complète terminée et visualisations sauvegardées.")

c:\billo\vegetation-evolution\.venv\Lib\site-packages\rasterio\__init__.py:378: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
C:\Users\billo.bah\AppData\Local\Temp\ipykernel_3164\2054967295.py:6: RuntimeWarning: invalid value encountered in divide
  ndvi = np.where((nir + red) == 0., 0, (nir - red) / (nir + red))


NDVI sauvegardé : ../ndvi/ndvi_cropped_straightened_images\1998-02-16_ndvi.TIF
NDVI sauvegardé : ../ndvi/ndvi_cropped_straightened_images\1998-03-04_ndvi.TIF
NDVI sauvegardé : ../ndvi/ndvi_cropped_straightened_images\1998-11-15_ndvi.TIF
NDVI sauvegardé : ../ndvi/ndvi_cropped_straightened_images\2007-01-08_ndvi.TIF
NDVI sauvegardé : ../ndvi/ndvi_cropped_straightened_images\2007-01-24_ndvi.TIF
NDVI sauvegardé : ../ndvi/ndvi_cropped_straightened_images\2007-02-09_ndvi.TIF
NDVI sauvegardé : ../ndvi/ndvi_cropped_straightened_images\2007-02-25_ndvi.TIF
NDVI sauvegardé : ../ndvi/ndvi_cropped_straightened_images\2007-03-13_ndvi.TIF
NDVI sauvegardé : ../ndvi/ndvi_cropped_straightened_images\2010-01-16_ndvi.TIF
NDVI sauvegardé : ../ndvi/ndvi_cropped_straightened_images\2010-02-01_ndvi.TIF
NDVI sauvegardé : ../ndvi/ndvi_cropped_straightened_images\2010-02-17_ndvi.TIF
NDVI sauvegardé : ../ndvi/ndvi_cropped_straightened_images\2010-12-02_ndvi.TIF
NDVI sauvegardé : ../ndvi/ndvi_cropped_straightened_